In [ ]:
# Packages / libraries
import os #provides functions for interacting with the operating system
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

# To install sklearn type "pip install numpy scipy scikit-learn" to the anaconda terminal

# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# Increases the size of sns plots
sns.set(rc={'figure.figsize':(8,6)})

#ARIMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

import pmdarima as pm
from pmdarima.model_selection import train_test_split

from pandas import to_datetime

import itertools
import warnings

import datetime
from datetime import datetime
warnings.filterwarnings('ignore')



#### Read  Data

In [ ]:
dataframe=pd.read_csv("Index2018.csv")

In [ ]:
df=dataframe.copy()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df['spx'].plot.hist(edgecolor='k')

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df.info

In [ ]:
df.columns

#### Plotting

In [ ]:
df.spx.plot(figsize=(20,5),color="Red")
plt.title('spx',size=24)
plt.show()

In [ ]:
df.spx.plot(figsize=(20,5))
df.rolling(window=7).mean()['spx'].plot(figsize=(20,5),c="Red")
plt.title('spx',size=24)
plt.show()

In [ ]:
df.plot(figsize=(20,5),)
plt.title('ALL',size=24)
plt.show()

In [ ]:
dataplot=sns.heatmap(df.corr(),cmap="YlGnBu",annot=True)
plt.show()

#### QQ plot

In [ ]:
import scipy.stats
import pylab

In [ ]:
scipy.stats.probplot(df.spx, plot = pylab)
plt.title("QQ Plot", size = 24)
pylab.show()

#### PreProcess

In [ ]:
df.date=pd.to_datetime(df.date,dayfirst=True)

In [ ]:
df=df.set_index("date")

In [ ]:
df=df.asfreq("b")

In [ ]:
df.isnull().sum()

In [ ]:
df.spx=df.spx.fillna(method="ffill")

In [ ]:
df.ftse=df.ftse.fillna(method="bfill")

In [ ]:
df.dax=df.dax.fillna(value=df.dax.mean())

In [ ]:
df.nikkei =df.nikkei.fillna(method="ffill")

In [ ]:
df.isnull().sum()

In [ ]:
df['market_value']=df.spx

In [ ]:
df=df.drop(["spx","dax","ftse","nikkei"],axis=1)

In [ ]:
df.head()

In [ ]:
df["market_value"].value_counts()

#### data inspection 

In [ ]:
import statsmodels
import statsmodels.tsa.stattools as sts

In [ ]:
sts.adfuller(df.market_value)

In [ ]:
#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(sales):
    result=sts.adfuller(sales)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [ ]:
adfuller_test(df.market_value)

In [ ]:
df1=df['1994-01-07':'1995-01-07']

In [ ]:
df1.head()

In [ ]:
df1.plot(figsize=(20,5),title="One year")
plt.show()
# df1.plot(style='k.',figsize=(20,5),title="One year")
# plt.show()

In [ ]:
# sesonality constant so we should use additive

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams

In [ ]:
s_dec_aditive=seasonal_decompose(df.market_value,model="additive")
rcParams['figure.figsize']=12,6
s_dec_aditive.plot()
plt.show()

In [ ]:
# s_dec_muli=seasonal_decompose(df.market_value,model="multiplicative")
# s_dec_muli.plot()
# plt.show()

In [ ]:
#significant or not

In [ ]:
import statsmodels.graphics.tsaplots as sgt

In [ ]:
sgt.plot_acf(df.market_value,lags=40,zero=False)
plt.title("ACF&MV")
plt.show()

In [ ]:
sgt.plot_pacf(df.market_value,lags=40,zero=False,method=("ols"))
plt.title("PACF&Rw")
plt.show()

#### to make dta stable

In [ ]:
from statsmodels.tsa.statespace.tools import diff

In [ ]:
df["diff_1"]=diff(df["market_value"],k_diff=1)

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
adfuller_test(df.diff_1)

In [ ]:
cor=df.corrwith(df.diff_1, axis=0, drop=False, method='pearson')
cor

In [ ]:
#plot Both of them

### Simple model 

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
MSE=mean_squared_error(df.market_value,df.diff_1)
print(np.sqrt(MSE))

#### splitting data

In [ ]:
size=int(len(df)*0.8)
df_train=df.iloc[:size]
df_test=df.iloc[size:]

In [ ]:
df_train.size,df_test.size

In [ ]:
df_test.shape

#### Simple Exponential Smoothing 

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
fitted_model=ExponentialSmoothing(df_train["market_value"],trend='mul',seasonal='mul',seasonal_periods=2).fit()

In [ ]:
test_prediction=fitted_model.forecast(1256)

In [ ]:
df_train["market_value"].plot(legend=True,label="train",figsize=(18,6))
df_test["market_value"].plot(legend=True,label="test",figsize=(18,6))
plt.show()

In [ ]:
df_train["market_value"].plot(legend=True,label="traint",figsize=(18,6))
df_test["market_value"].plot(legend=True,label="test",figsize=(18,6))
test_prediction.plot(legend=True,label="pred")
plt.show()

In [ ]:
df_train["market_value"].plot(legend=True,label="traint",figsize=(18,6))
df_test["market_value"].plot(legend=True,label="test",figsize=(18,6))
test_prediction.plot(legend=True,label="pred",xlim=["2013-04-08 ","2018-01-29"])
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
MSE1=mean_squared_error(df_test["market_value"],test_prediction)
print(np.sqrt(MSE1))

In [ ]:
MAE=mean_absolute_error(df_test["market_value"],test_prediction)
MAE

In [ ]:
final_model=ExponentialSmoothing(df["market_value"],trend='mul',seasonal='mul',seasonal_periods=2).fit()

In [ ]:
test_prediction1=final_model.forecast(360)

In [ ]:
df['market_value'].plot(figsize=(18,4))
test_prediction1.plot()
plt.show()

### AutoRegression

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df['market_value'])
plt.show()

In [ ]:
from statsmodels.tsa.ar_model import AR,ARResults

In [ ]:
start=len(df_train)
end=len(df_train) + len(df_test)-1

In [ ]:
model=AR(df_train['market_value'])

In [ ]:
AR_fit=model.fit(maxlag=1)

In [ ]:
AR_fit.params

In [ ]:
prediction=AR_fit.predict(start=start, end=end)

In [ ]:
df_test.head()

In [ ]:
df_test['market_value'].plot(legend=True,figsize=(16,4),label="test")
prediction.plot(legend=True,label="predict")
plt.show()

In [ ]:
model2=AR(df_train['market_value'])
AR_fit2=model2.fit(maxlag=2)
prediction2=AR_fit2.predict(start=start, end=end)
df_test['market_value'].plot(legend=True,figsize=(16,4),label="test")
prediction.plot(legend=True,label="predict")
prediction2.plot(legend=True,label="predict2")
plt.show()

In [ ]:
model3=AR(df_train['market_value'])

In [ ]:
ARfit=model3.fit(ic='t-stat')

In [ ]:
ARfit.params

In [ ]:
model4=AR(df_train['market_value'])
AR_fit4=model4.fit(maxlag=28)
prediction4=AR_fit4.predict(start=start, end=end)
df_test['market_value'].plot(legend=True,figsize=(16,4),label="test")
prediction.plot(legend=True,label="predict")
prediction2.plot(legend=True,label="predict2")
prediction4.plot(legend=True,label="predict4")
plt.show()

In [ ]:
label=['AR1','AR2','AR28']
prds=[prediction,prediction2,prediction4]
for i in range(3):
    error=np.sqrt(mean_squared_error(df_test["market_value"],prds[i]))
    print(f'{label[i]} MSE was: {error}')

In [ ]:
#Forecasting

In [ ]:
final_model=AR(df['market_value'])
AR_fit=final_model.fit()
forcast_value=AR_fit.predict(start=len(df), end=len(df)+360)

In [ ]:
df['market_value'].plot(figsize=(14,4))
forcast_value.plot()
plt.plot()

##### causalitytests

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

In [ ]:
grangercausalitytests(df[["market_value","diff_1"]],maxlag=3)

In [ ]:
#Month

In [ ]:
# from statsmodels.graphics.tsaplots import month_plot,quarter_plot,
# quarter_plot(df["market_value"]);

In [ ]:
from statsmodels.tsa.arima_model import ARMA

In [ ]:
model_ar = ARMA(df_train.market_value, order=(1,0))
results_ar = model_ar.fit()
results_ar.summary()

In [ ]:
predict=results_ar.predict(start=start, end=end)
df_test['market_value'].plot(legend=True,figsize=(16,4),label="test")
predict.plot(legend=True,label="predict")
plt.show()

In [ ]:
# ARfit=model3.fit(ic='t-stat')
# ARfit.params

In [ ]:
model_ar_1 = ARMA(df.market_value, order=(1,0))
results_ar_1 = model_ar_1.fit()
results_ar_1.summary()

In [ ]:
model_ar_2 = ARMA(df.market_value, order=(2,0))
results_ar_2 = model_ar_2.fit()
results_ar_2.summary()

#### LLR test

In [ ]:
def LLR_test(mod_1, mod_2, DF=1):
    L1 = mod_1.fit().llf
    L2 = mod_2.fit().llf
    LR = (2*(L2-L1))
    p = chi2.sf(LR, DF).round(3)
    return p

In [ ]:
LLR_test(model_ar_1, model_ar_2)

In [ ]:
print("LLR test: " + str(LLR_test(model_ar, model_ar_7, DF = 6)))
# if you comparing this you should increase your DF

#### ARIMA 

In [ ]:
model_arima = ARIMA(df_train.market_value, order=(1,1,1))
results = model_arima.fit()
results.summary()

In [ ]:
# Obtain predicted values
start=len(df_train)
end=len(df_train)+len(df_test)-1
predictions = results.predict(start=start, end=end, dynamic=False, typ='levels').rename('ARIMA(1,1,1) Predictions')

In [ ]:
##Plot

In [ ]:
# Compare predictions to expected values
for i in range(len(predictions)):
    print(f"predicted={predictions[i]:<11.10}, expected={df_test['market_value'][i]}")

In [ ]:
# Plot predictions against known values
title = 'Real Manufacturing and Trade Inventories'
ylabel='Chained 2012 Dollars'
xlabel='' # we don't really need a label here

ax = df_test['market_value'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel)


In [ ]:
from sklearn.metrics import mean_squared_error

error = mean_squared_error(df_test['market_value'], predictions)
print(f'ARIMA(1,1,1) MSE Error: {error:11.10}')

In [ ]:
###auto arima

In [ ]:
stepwise_fit = auto_arima(df1['Thousands of Passengers'], start_p=1, start_q=1,
                          max_p=3, max_q=3, m=12,
                          start_P=0, seasonal=True,
                          d=None, D=1, trace=True,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True)           # set to stepwise

stepwise_fit.summary()

#### Forcast

In [ ]:
model = ARIMA(df.market_value,order=(1,1,1))
results = model.fit()
fcast = results.predict(len(df),len(df)+360,typ='levels').rename('ARIMA(1,1,1) Forecast')

In [ ]:
# Plot predictions against known values
title = 'Real Manufacturing and Trade Inventories'
ylabel='Chained 2012 Dollars'
xlabel='' # we don't really need a label here

ax = df.market_value.plot(legend=True,figsize=(12,6),title=title)
fcast.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel)
